<a id="part0"></a>
# 0.Abstract:

In this analysis after numerically understand data.I visualized the dataset and tried to understand relations between columns.And i dropped tenure column after EDA. I transformed categorical values into numerical.Gender column label encoded and Geography column OneHotEncoded.To perform neural network better. I applied min max scaling to data.And I split dataset into 3 parts,Training ,Test and validation.NN model has 1 input layer and 5 hidden layer.I experimented a little bit at neural network architecture.Model i used gave best result in my experiences. I also tried Logistic Regression ,SVC and Random Forest Classifier. Because of our dataset is imbalanced. Result of Logistic regression and SVC are bad. But Random Forest Classifier gave almost same results with neural network.

**Used Libraries:**
* 1. Pandas
* 2. Numpy
* 3. Seaborn
* 4. Matplotlib
* 5. Sklearn
* 6. Tensorflow

![](https://ml8ygptwlcsq.i.optimole.com/fMKjlhs.f8AX~1c8f3/w:1200/h:675/q:auto/https://www.unite.ai/wp-content/uploads/2019/11/artificial-neural-network-3501528_1920.png)

**Table of contents:**

* [0.Abstract:](#part0)
* [1.Understanding Dataset:](#part1)
* [2.Exploratory Data Analysis:](#part2)
* [3.Encoding:](#part3)
* [4.Normalization(minmax scaling) and Splitting the dataset:](#part4)
* [5.Building the Neural Network Model:](#part5)
* [6.Training the Neural Network:](#part6)
* [7.Traditional Methods:](#part7)
* [8.Conclusion:](#part8)

<a id="part1"></a>
# 1.Understand the Dataset:

Lets understand data with numbers.


In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import seaborn as sns # For visualization
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("/kaggle/input/churn-modelling/Churn_Modelling.csv")
data.head()

In [ ]:
data.describe().T

In [ ]:
data.shape

Dataset has 10.000 rows and 14 columns.For training we don't need "RowNumber","CustomerId" and "Surname" columns. Lets start with dropping these columns.

In [ ]:
data.drop(columns = ["RowNumber","CustomerId","Surname"], inplace = True) #3 Columns dropped.
data.head()

Second, Lets check balance of data and NaN values.

In [ ]:
data.Exited.value_counts(),data.isnull().sum()

Seems like dataset has no Nan values but dataset is unbalanced. %80 of target value is 0.We can balance it but we are going to use neural network for training.We don't need to balance dataset.

<a id="part2"></a>
# 2.Exploratory Data Analysis:
Lets discover relations between Exited and rest of the data.

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(data.corr(),annot = True)

In [ ]:
fig,ax = plt.subplots(nrows=2, ncols=2 , figsize=(25,10))
sns.barplot(x= pd.cut(data.CreditScore,bins = 5).unique().sort_values(),y = data.groupby(pd.cut(data.CreditScore,bins = 5)).mean()["Exited"], ax = ax[0,0]),ax[0,0].set(xlabel='Credit Score')
sns.barplot(x=np.sort(data.Geography.unique()),y = data.groupby(data.Geography).mean()["Exited"],ax = ax[0,1])
sns.barplot(x=np.sort(data.Gender.unique()),y = data.groupby(data.Gender).mean()["Exited"],ax = ax[1,0])
sns.barplot(x= pd.cut(data.Age,bins = 20).unique().sort_values(),y = data.groupby(pd.cut(data.Age,bins = 20)).mean()["Exited"],ax = ax[1,1]),ax[1,1].set(xlabel='Customer Age')

Seems like lower credit score means company is more likely to lose customer. Also being at age range between 45-65 means also company is going to lose customer.At figure [1,0] male customers seems like more loyal than female customers.Let's dig a little bit more.

In [ ]:
fig,ax1 = plt.subplots(nrows=2, ncols=2 , figsize=(25,10))
sns.barplot(x = data.Tenure.unique(),y = data.groupby(data.Tenure).mean()["Exited"] , ax = ax1[0,0]),ax1[0,0].set(xlabel='Tenure')
sns.barplot(x = data.NumOfProducts.unique(),y = data.groupby(data.NumOfProducts).mean()["Exited"]*100 , ax = ax1[0,1]),ax1[0,1].set(xlabel="Number Of Products")
sns.boxplot(data.Balance, ax = ax1[1,0], color = "lime"),ax1[1,0].set(xlabel="Checking for outliers at Balance column")
sns.barplot(x = pd.cut(data.Balance , bins = 10).unique().sort_values(),y = data.groupby(pd.cut(data.Balance, bins = 10)).mean()["Exited"]*100, ax=ax1[1,1]),ax1[1,1].set(xlabel="Balance column's customer loss percent")

At first graph, Company customer losses has no relation with customer membership duration. We can drop this column for better training.Second graph sayin, customers who has 2 and 4 products is more likely to exit.Last 2 figures about balance column. First one is for outliers at balance column and second one is customer loss rate.Customers with balance of +200k, have higher chance to exit.

In [ ]:
data.drop(columns = ["Tenure"],inplace = True)# I dropped "Tenure" column here.
fig,ax2 = plt.subplots(nrows=2, ncols=2 , figsize=(25,10))
sns.barplot(x = data.HasCrCard.unique(),y = data.groupby(data.HasCrCard).mean()["Exited"] , ax = ax2[0,0]),ax2[0,0].set(xlabel='Has Credit Card')
sns.barplot(x = data.IsActiveMember.unique(),y = data.groupby(data.IsActiveMember).mean()["Exited"], ax = ax2[0,1]),ax2[0,1].set(xlabel="Is Active Member Column")
sns.boxplot(data.EstimatedSalary, ax = ax2[1,0], color = "purple"),ax2[1,0].set(xlabel="Checking for outliers at Estimated Salary column")
sns.barplot(x = pd.cut(data.EstimatedSalary , bins = 10).unique().sort_values(),y = data.groupby(pd.cut(data.EstimatedSalary, bins = 10)).mean()["Exited"]*100, ax=ax2[1,1]),ax2[1,1].set(xlabel="Estimated Salary column's customer loss percent")

"HasCrCard" and "EstimatedSalary" columns have no pattern at "Exited" column.We might drop these columns too. Active members chance of leaving company is double times.

In [ ]:
#data.drop(columns = ["HasCrCard","EstimatedSalary"],inplace = True)

When i dropped these columns its effectted NN negatively but randomforest worked same.So i decided to not drop.

<a id="part3"></a>
# 3.Encoding:

In dataset we have 2 categorical columns. For geography column we're going to apply OneHotEncoding and use Label encoder for the gender column.

In [ ]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

data.Gender = LabelEncoder().fit_transform(data.Gender) # Gender label encoding
data = pd.concat((pd.DataFrame(OneHotEncoder(sparse = False).fit_transform(pd.DataFrame(data.Geography)),columns = ["FromFrance","FromSpain","FromGermany"]),data),axis = 1) #Geography onehotencoding
data.drop(columns= ["Geography"],inplace = True)
data.head()

Good dataset looks better now.Before training one last thing left to do. Lets scale Balance, CreditScore and EstimatedSalary columns.Because we're going to perform Neural Network for training and NN works better with scaled data.I'am going to apply MinMaxScaler to our data for better neural network training.

<a id="part4"></a>
# 4.Normalization(minmax scaling) and Split the dataset:

In [ ]:
from sklearn.preprocessing import MinMaxScaler
data[["CreditScore","Age","Balance","EstimatedSalary"]] = MinMaxScaler(copy = False).fit_transform(data[["CreditScore","Age","Balance","EstimatedSalary"]])
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
X = data.iloc[:,:-1]
y = data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25,random_state = 10)
valitrain, X_test, valitest, y_test = train_test_split(X_test,y_test,test_size = 0.5)

We finished preprocessing stage.Now We're going to create a neural network and use it for training.

<a id="part5"></a>
# 5.Building the Neural Network Model:

In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(32 , activation = "relu",input_shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1,activation = "sigmoid")
])
model.summary()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),loss = tf.losses.BinaryCrossentropy(),metrics= ["accuracy",'binary_crossentropy'])

<a id="part6"></a>
# 6.Training the Neural Network:

In [ ]:
history = model.fit(X_train, y_train , validation_data = (valitrain,valitest),epochs = 100)

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.round_(y_pred)
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_test,y_pred)
cm1

In [ ]:
model.evaluate(X_test, y_test,batch_size=10)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

85 percent accuracy not too bad. But recall percentage = 0.55 means we are predicting lost customers with %55 accuracy.

In [ ]:
hist = pd.DataFrame(history.history)

Lets visualize results.

In [ ]:
f, ax = plt.subplots(1, 1,figsize = (15,8))
sns.lineplot(x = range(0,100),y = hist.binary_crossentropy , color = "green",label='Training binary crossentropy')
sns.lineplot(x = range(0,100),y = hist.val_binary_crossentropy , color = "blue",label='Validation binary crossentropy')
sns.lineplot(x = range(0,100),y = hist.accuracy , color = "red",label='Training Accuracy')
sns.lineplot(x = range(0,100),y = hist.val_accuracy , color = "purple",label='Validation Accuracy')


Seems like 30 epochs is better for training.

<a id="part7"></a>
# 7.Traditional Methods:

I also want to try some traditional ML methods like SVC ,Random Forest and Logistic Regression.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
le = LogisticRegression()
svc = SVC()
rf = RandomForestClassifier()

In [ ]:
le.fit(X_train,y_train)
svc.fit(X_train,y_train)
rf.fit(X_train,y_train)

In [ ]:
y_predle = le.predict(X_test)
y_predsvc = svc.predict(X_test)
y_predrf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test,y_predle))

In [ ]:
print(classification_report(y_test,y_predsvc))

In [ ]:
print(classification_report(y_test,y_predrf))

In [ ]:
cmrf = confusion_matrix(y_test, y_predrf)
cmsvc = confusion_matrix(y_test, y_predsvc)
cmrf , cm1 , cmsvc

<a id="part8"></a>
# 8.Conclusion:

Random Forest Classifier and ANN gave us best results.Svc and Logistic regression seems failed a little bit.That because we are dealing with imbalanced dataset.To improve SVC and LogReg results.We can try smote for upsampling and train again.But i'm not going to do that. If you want to learn how to implement upsampling you can check my other notebooks.I tried couple things at ANN but i can't improve the results and ı don't know what to expect accuracy at loss customers. So do you have any suggestions to improve ANN result. Thanks for your time.